### Import

In [1]:
import sys
import os
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random

import json
import time
import pandas
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor
import torchvision.transforms as tt
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from torchvision import transforms, utils, datasets, models
from torch.autograd import Variable
# from keras.optimizers import Adam
import glob
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import pathlib
from pyspark.sql.functions import expr

from tiktok_downloader import snaptik
import os
import torch
from transformers import VideoMAEImageProcessor, VideoMAEForVideoClassification
import pytorchvideo.data
from torch.utils.data import DataLoader
import numpy as np

from pyspark.sql.functions import col, from_json, json_tuple, udf, explode
from pyspark.sql.types import StructType, StructField, StringType, MapType, ArrayType

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    RemoveKey,
    ShortSideScale,
    UniformTemporalSubsample,
)
from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize,
)

from transformers import AutoImageProcessor, TimesformerForVideoClassification

### Init spark

In [2]:
scala_version = '2.12'  # your scala version
spark_version = '3.5.1' # your spark version
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.7.0' #your kafka version
]
    # spark = SparkSession.builder.master("local").appName("TikTokVideos").config("spark.jars.packages", ",".join(packages)).getOrCreate()
    # spark

spark = SparkSession \
    .builder \
    .appName("Spark-TikTokVideos") \
    .master("local[*]") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "8g") \
    .config("spark.python.worker.reuse",True) \
    .config("spark.sql.execution.arrow.pyspark.enabled", True) \
    .config("spark.jars.packages", ",".join(packages)) \
    .getOrCreate()

spark

:: loading settings :: url = jar:file:/teamspace/studios/this_studio/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/zeus/.ivy2/cache
The jars for the packages stored in: /home/zeus/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-68a14eaa-db12-491d-8f21-dc8b78cde6f9;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
	found org.apache.kafka#kafka-clients;3.7.0 in central
	found com.github.luben#zstd-jni;1.5.5-6 in central
	f

### Setup model & Spark streaming with Kafka

#### Model setup

In [3]:
id2label = {0: 'Adult Content', 1: 'Harmful Content', 2: 'Safe', 3: 'Suicide'}
label2id = {'Adult Content': 0, 'Harmful Content': 1, 'Safe': 2, 'Suicide': 3}
device = "cpu"
model_ckpt = "/teamspace/studios/this_studio/checkpoint_model/timesformer-base-finetuned-k400-finetuned-ucf101-subset/checkpoint-2300"

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
val_transform = Compose(
    [
        ApplyTransformToKey(
            key="video",
            transform=Compose(
                [
                    UniformTemporalSubsample(16),
                    Lambda(lambda x: x / 255.0),
                    RandomShortSideScale(min_size=256, max_size=320),
                    Normalize(mean, std),
                    Resize((224, 224)),
                ]
            ),
        ),
    ]
)
    #image_processor = VideoMAEImageProcessor.from_pretrained(model_ckpt, attn_implementation="sdpa")
model = TimesformerForVideoClassification.from_pretrained(
    model_ckpt,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,
).to(device)

In [4]:
def prediction_label(topic_name, spark_df):
    def collate_fn(examples):
        # permute to (num_frames, num_channels, height, width)
        pixel_values = torch.stack(
             [example["video"].permute(1, 0, 2, 3) for example in examples]
        )
        return {"pixel_values": pixel_values}

    def predict_batch_fn():

        import pytorchvideo.data
        import numpy as np
        import random
        import os
        import socket
        import shutil
        
        def preprocess_batch(tiktok_urls):

            idx_folder = socket.gethostname() + str(random.randint(1, 100))
            download_folder = f"/teamspace/studios/this_studio/KidGuardScan/video_predict/{idx_folder}/noclass"
            os.makedirs(download_folder, exist_ok=True)
            
            for index, url in enumerate(tiktok_urls):
                d = snaptik(url)
                parts = url.split("/")
                username = parts[3][1:]
                video_id = parts[5]
                name_video = f"{index}.mp4"
                # download_folder = (
                #     "/teamspace/studios/this_studio/Doan/video_predict/{idx_folder}/noclass"
                # )
                download_path = os.path.join(download_folder, name_video)
                # if os.path.exists(download_path):
                #     continue
                try:
                    d[0].download(download_path)
                except:
                    print("Error: ", url)

            val_dataset = pytorchvideo.data.Ucf101(
                data_path=os.path.join(
                    "/teamspace/studios/this_studio/KidGuardScan/video_predict", str(idx_folder)
                ),
                clip_sampler=pytorchvideo.data.make_clip_sampler(
                    "random", 2),  # Note the positional arguments
                decode_audio=False,
                transform=val_transform,
                video_sampler=torch.utils.data.SequentialSampler,
            )
            return val_dataset, idx_folder

        def predict(inputs: np.ndarray) -> np.ndarray:
            decoded_urls = [url.decode("utf-8") for url in inputs]
            inputs_dataset, idx_folder = preprocess_batch(decoded_urls)
            size_prediction = len(inputs)
            dataloader = DataLoader(
                inputs_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn
            )
            predictions = []
            for i, batch in enumerate(dataloader):
                if i == size_prediction:
                    break

                pixel_values = batch["pixel_values"].to(device)
                with torch.no_grad():
                    outputs = model(pixel_values)
                    logits = outputs.logits  # [batch_size, num_classes]
                    prediction = torch.argmax(logits, dim=-1).item()
                predictions.append(prediction)

            base_folder = "/teamspace/studios/this_studio/KidGuardScan/video_predict/"
            shutil.rmtree(os.path.join(base_folder, idx_folder))

            return np.array(predictions)

        return predict

    from pyspark.ml.functions import predict_batch_udf
    from pyspark.sql.types import FloatType, IntegerType

    predict_fn_for_model = predict_batch_udf(predict_batch_fn,
                                    return_type=IntegerType(),
                                    batch_size=3,
    )

    result = spark_df.withColumn("predicted_label_id", predict_fn_for_model(col("encoded_urls"))) \
                    .withColumn("predicted_label", when(col("predicted_label_id") == 0, "Adult Content")
                                   .when(col("predicted_label_id") == 1, "Harmful Content")
                                   .when(col("predicted_label_id") == 2, "Safe")
                                   .when(col("predicted_label_id") == 3, "Suicide")
                                   .otherwise("Unknown"))  # Handle unexpected values
    
    return result

#### Kafka setup

In [5]:
topic_name = "Test_with_MongoDB11"
kafka_server = 'localhost:9092'

In [6]:
kafkaDf = spark.readStream.format('kafka').option('kafka.bootstrap.servers', kafka_server).option('subscribe', topic_name).option('startingOffsets', 'latest').load()
kafkaDf = kafkaDf.withColumn("value", col("value").cast("string"))
kafkaDf = kafkaDf.withColumn("key", col("key").cast("string"))
# kafkaDf = kafkaDf.filter(col("key").isin(hashtag))
kafkaDf = kafkaDf.withColumn("tiktok_url",
    expr("concat('https://www.tiktok.com/@', get_json_object(value, '$.author.uniqueId'), '/video/', get_json_object(value, '$.id'))"))
kafkaDf = kafkaDf.withColumn("encoded_urls", expr("unhex(hex(tiktok_url))"))

kafkaDf = prediction_label(topic_name, kafkaDf)

value_schema = StructType([
    StructField("id", StringType(), True),
    StructField("desc", StringType(), True),
    StructField("author", StructType([
        StructField("uniqueId", StringType(), True)
    ])),
    StructField("textExtra", ArrayType(MapType(StringType(), StringType())), True),
    StructField("stats", MapType(StringType(), StringType()), True),
    StructField("diversificationLabels", ArrayType(StringType()), True),
    StructField("suggestedWords", ArrayType(StringType()), True),
    StructField("createTime", StringType(), True),
    StructField("locationCreated", StringType(), True),
    StructField("video", MapType(StringType(), StringType()), True)
])

# Extract the JSON fields from the 'value' column
kafkaDf = kafkaDf.withColumn("value_json", from_json(col("value"), value_schema))

# Define a UDF to concatenate hashtags
@udf(returnType=StringType())
def concatenate_hashtags(textExtra):
    if textExtra is None:
        return ""
    return " ".join([item['hashtagName'] for item in textExtra if item['hashtagName']])

# Create the new DataFrame with the required columns
kafkaDf_predict = kafkaDf.select(
    col("value_json.id").alias("id"),
    col("value_json.desc").alias("description"),
    col("value_json.author.uniqueId").alias("user_id"),
    concatenate_hashtags(col("value_json.textExtra")).alias("hashtags"),
    col("value_json.stats").alias("stats"),
    col("value_json.diversificationLabels").alias("diversificationLabels"),
    col("value_json.suggestedWords").alias("suggestedWords"),
    col("value_json.createTime").alias("created_at"),
    col("value_json.video.duration").alias("duration"),
    col("value_json.locationCreated").alias("locationCreated"),
    col("tiktok_url"),
    # col("encoded_urls"),
    # col("key"),
    col("predicted_label_id"),
    col("predicted_label")
)

### Streaming setup and write the result (predicted label and metadata videos) to mongodb

In [7]:
with open('KidGuardScan/uri_mongodb.txt', 'r') as file:
    uri = file.read().strip()

In [8]:
# Import necessary libraries
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi


# Prompt the user to enter the MongoDB URI
# uri = getpass("Enter the MongoDB URI: ")  ##read from files, ...

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [9]:
db = client['VideoTikTok']
collection = db['videos']

In [10]:
def write_to_mongo(df, epoch_id):
    if df.isEmpty():
        print(f"Batch {epoch_id} is empty. No records to write to MongoDB.")
        return
            
    data = df.toJSON().collect()
    data = [json.loads(doc) for doc in data]
    
    for doc in data:
        doc['_id'] = doc.pop('id')
    
    collection.insert_many(data)
    # client.close()
    print(f"Batch {epoch_id} written to MongoDB successfully with {len(data)} records.")

In [11]:
q1name = "prediction"
stream_writer1 = kafkaDf_predict.writeStream.queryName(q1name).trigger(processingTime="120 seconds").outputMode("append").foreachBatch(write_to_mongo).start()

24/07/05 05:23:10 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-4af7e4ae-718c-4fd1-9b93-deb5839cc243. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/07/05 05:23:10 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Batch 0 is empty. No records to write to MongoDB.


In [12]:
try:
    stream_writer1.awaitTermination()
except KeyboardInterrupt:
    print("Stream stopped.")

24/07/05 05:28:02 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
24/07/05 05:28:03 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
24/07/05 05:28:03 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
24/07/05 05:28:52 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
24/07/05 05:28:53 WARN KafkaDataConsumer: KafkaDataConsumer is not running in UninterruptibleThread. It may hang when KafkaDataConsumer's methods are interrupted because of KAFKA-1894
24/07/05 05:28:53 WARN KafkaDataConsumer: KafkaDataConsumer is not running in Un

Batch 1 written to MongoDB successfully with 3 records.
